# 任务1：对数据进行探索和分析 5.11-5.12
数据集是金融数据（非原始数据，已经处理过了），我们要做的是预测贷款用户是否会逾期。
表格中 "status" 是结果标签：0表示未逾期，1表示逾期。

要求：
1. 数据切分方式 - 三七分，其中测试集30%，训练集70%，随机种子设置为2018
2. 数据类型的分析
3. 无关特征删除
4. 数据类型转换
5. 缺失值处理
6. 其他可借鉴的数据分析处理

In [1]:
import numpy as np
import pandas as pd

## 一. 读取数据
读取任务1中csv文件是要将编码格式制定成gbk；
中文字符的Unicode编码0x0800-0xFFFF之间,(utf-8包含了部分汉字)
当你试图将该“中文字符”转成U码的utf-8时超出了其范筹
而GBK 规范收录了ISO 10646.1中的全部 CJK 汉字和符号，并有所补充，
所以解决方法是将.decode('utf-8')改为.decode('gbk')
参考：https://blog.csdn.net/stone9159/article/details/79071316

In [2]:
# 导入数据
data = pd.read_csv(r"task1_data.csv",encoding="gbk")
# 将显示的最大列数设置成100，以展示全部列
pd.set_option("max_columns", 100)
# 抽样查看数据
data.sample(10)

,Unnamed: 0,custid,trade_no,bank_card_no,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,trans_days_interval,regional_mobility,student_feature,repayment_capability,is_high_user,number_of_trans_from_2011,first_transaction_time,historical_trans_amount,historical_trans_day,rank_trad_1_month,trans_amount_3_month,avg_consume_less_12_valid_month,abs,top_trans_count_last_1_month,avg_price_last_12_month,avg_price_top_last_12_valid_month,reg_preference_for_trad,trans_top_time_last_1_month,trans_top_time_last_6_month,consume_top_time_last_1_month,consume_top_time_last_6_month,cross_consume_count_last_1_month,trans_fail_top_count_enum_last_1_month,trans_fail_top_count_enum_last_6_month,trans_fail_top_count_enum_last_12_month,consume_mini_time_last_1_month,max_cumulative_consume_later_1_month,max_consume_count_later_6_month,railway_consume_count_last_12_month,pawns_auctions_trusts_consume_last_1_month,pawns_auctions_trusts_consume_last_6_month,jewelry_consume_count_last_6_month,status,source,first_transaction_day,trans_day_last_12_month,id_name,apply_score,apply_credibility,query_org_count,query_finance_count,query_cash_count,query_sum_count,latest_query_time,latest_one_month_apply,latest_three_month_apply,latest_six_month_apply,loans_score,loans_credibility_behavior,loans_count,loans_settle_count,loans_overdue_count,loans_org_count_behavior,consfin_org_count_behavior,loans_cash_count,latest_one_month_loan,latest_three_month_loan,latest_six_month_loan,history_suc_fee,history_fail_fee,latest_one_month_suc,latest_one_month_fail,loans_long_time,loans_latest_time,loans_credit_limit,loans_credibility_limit,loans_org_count_current,loans_product_count,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
1451,3768,3500912,20180504180919968000000021285352,卡号1,0.02,0.98,0,1.16,1.00,0.351,18.0,27.0,14.0,2.0,NaN,11580,0,11.0,20170412.0,159990,116.0,0.30,42070,6.0,2370,0.05,970,0.55,一线城市,6.0,9.0,6.0,9.0,1.0,0.0,2.0,3.0,1023.0,1460,5.0,0.0,2370,27030,0.0,0,xs,404.0,82.0,高婷婷,545.0,76.0,19.0,6.0,5.0,29.0,2018-04-29,13.0,19.0,21.0,510.0,77.0,48.0,45.0,3.0,21.0,11.0,10.0,3.0,8.0,20.0,71.0,6.0,3.0,0.0,357.0,2018-04-29,2400.0,74.0,10.0,10.0,7900.0,2520.0,6300.0,78.0,11.0,13.0,13200.0,5900.0,5.0,5.0
881,2215,1748390,20180507115512144000000388208649,卡号1,0.00,1.00,0,0.57,0.58,0.195,8.0,15.0,54.0,3.0,NaN,8960,0,15.0,20150924.0,36530,54.0,1.00,0,5.0,1840,1.00,1280,NaN,一线城市,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,0.0,0.0,0,0,0.0,0,xs,970.0,203.0,邓根元,515.0,71.0,6.0,4.0,1.0,6.0,2018-04-24,4.0,4.0,4.0,481.0,75.0,35.0,28.0,2.0,14.0,2.0,12.0,0.0,0.0,6.0,29.0,15.0,0.0,0.0,357.0,2018-01-30,2000.0,75.0,12.0,12.0,2900.0,1625.0,4700.0,77.0,2.0,2.0,7800.0,4400.0,13.0,97.0
4402,11126,1959805,20180507120652538000000388249142,卡号1,0.00,1.00,0,0.30,0.92,0.069,6.0,73.0,72.0,1.0,NaN,12050,0,11.0,20170327.0,20690,23.0,0.75,3040,7.0,550,1.00,1080,0.50,境外,0.0,28.0,0.0,28.0,0.0,0.0,2.0,2.0,0.0,500,2.0,0.0,0,0,0.0,1,xs,420.0,93.0,汤鸣勋,611.0,73.0,11.0,7.0,4.0,13.0,2018-05-04,2.0,7.0,10.0,583.0,74.0,3.0,2.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,284.0,2017-11-03,1100.0,72.0,1.0,1.0,1100.0,1100.0,1000.0,78.0,1.0,1.0,1000.0,1000.0,3.0,185.0
2910,7293,2856413,20180507123449145000000388365370,卡号1,0.03,0.85,1500,0.65,0.57,0.269,13.0,30.0,25.0,3.0,1.0,2060,0,23.0,20141120.0,46260,117.0,0.80,3450,7.0,2190,1.00,780,0.60,二线城市,0.0,30.0,0.0,32.0,0.0,1.0,3.0,3.0,0.0,340,2.0,0.0,0,0,0.0,0,xs,1278.0,82.0,周俊杉,543.0,73.0,6.0,2.0,3.0,6.0,2018-04-25,1.0,2.0,4.0,504.0,72.0,18.0,15.0,1.0,9.0,4.0,5.0,0.0,3.0,7.0,18.0,4.0,0.0,0.0,308.0,2018-03-23,1300.0,71.0,5.0,5.0,2300.0,1300.0,2900.0,73.0,4.0,4.0,3300.0,1925.0,12.0,45.0
4516,11391,1454329,20180507115423252000000388204010,卡号1,0.00,1.00,200,0.87,0.59,0.463,20.0,28.0,20.0,2.0,1.0,24

## 二. 数据分析与处理
1. 数据类型、重复值分析
2. 无关特征删除、类型转换
3. 缺失值处理
4. 其他可借鉴的数据分析处理

### 1. 数据类型分析

In [3]:
data.info()
data.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4754 entries, 0 to 4753
Data columns (total 90 columns):
Unnamed: 0                                    4754 non-null int64
custid                                        4754 non-null int64
trade_no                                      4754 non-null object
bank_card_no                                  4754 non-null object
low_volume_percent                            4752 non-null float64
middle_volume_percent                         4752 non-null float64
take_amount_in_later_12_month_highest         4754 non-null int64
trans_amount_increase_rate_lately             4751 non-null float64
trans_activity_month                          4752 non-null float64
trans_activity_day                            4752 non-null float64
transd_mcc                                    4752 non-null float64
trans_days_interval_filter                    4746 non-null float64
trans_days_interval                           4752 non-null float64
regional_mobility

(4754, 90)

### 2. 重复值分析

In [4]:
# 检查是否要重复数据
data.duplicated().any()

False

In [5]:
# 检查是否用重复的用户，结果False
data["custid"].duplicated().any()

False

### 3. 无关特征删除、类型转换

#### 3.1 object类型的数据

In [6]:
data.select_dtypes(include="object").columns

Index(['trade_no', 'bank_card_no', 'reg_preference_for_trad', 'source',
       'id_name', 'latest_query_time', 'loans_latest_time'],
      dtype='object')

In [7]:
'''分析object类型数据'''
# 检查是否有重复的trade_no，结果False，且交易的单号不影响结果需要删除
# display(data["trade_no"].duplicated().any())
# 检查bank_card_no，全为相同内容，需要删除
display(data["bank_card_no"].value_counts())
# 检查reg_preference_for_trad，分为5大类需要使用LabelEncoder编码
display(data["reg_preference_for_trad"].value_counts())
# 检查source，全为相同内容，需要删除
display(data["source"].value_counts())
# loans_latest_time、latest_query_time 时间内容需要调整
#id_name为用户姓名不影响结果需要删除

卡号1    4754
Name: bank_card_no, dtype: int64

一线城市    3403
三线城市    1064
境外       150
二线城市     131
其他城市       4
Name: reg_preference_for_trad, dtype: int64

xs    4754
Name: source, dtype: int64

In [8]:
'''处理object类型数据'''
#删除trade_no、bank_card_no、source、id_name、source列
data.drop(columns=["trade_no","bank_card_no","id_name","source"], axis=1, inplace=True)
data.select_dtypes(include="object").columns

Index(['reg_preference_for_trad', 'latest_query_time', 'loans_latest_time'], dtype='object')

In [9]:
#LabelEncoder编码处理reg_preference_for_trad
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
data["reg_preference_for_trad"] = lb.fit_transform(data["reg_preference_for_trad"].astype(str))
data["reg_preference_for_trad"].value_counts()

1    3403
2    1064
5     150
3     131
4       4
0       2
Name: reg_preference_for_trad, dtype: int64

In [10]:
# 将'latest_query_time', 'loans_latest_time'转换成日期格式，然后计算与当前日期的天数差值
data['latest_query_time'] = (pd.to_datetime(data['latest_query_time']) - pd.to_datetime('today')).apply(lambda col: col.days)
data['loans_latest_time'] = (pd.to_datetime(data['loans_latest_time']) - pd.to_datetime('today')).apply(lambda col: col.days)

#### 3.2 非object类型的数据

In [11]:
# Unnamed: 0 为序号需要删除，custid为用户id需要删除
data.drop(columns=["Unnamed: 0","custid"], axis=1, inplace=True)

### 4. 缺失值处理

In [12]:
data.isnull().sum().value_counts()

297     30
2       15
0       12
304     11
8        6
16       3
12       2
104      1
2998     1
426      1
26       1
3        1
dtype: int64

In [13]:
data.isnull().sum().argmax()

/Users/shenqi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  """Entry point for launching an IPython kernel.


'student_feature'

In [14]:
#找出缺失值最多的特征，经检查为student_feature，预估含义是用户的学生类型，考虑将空值设成0
data["student_feature"].fillna(value=0,inplace=True)

In [15]:
data.isnull().sum(axis=1).value_counts()

0     3983
1      385
41     249
2       37
42      27
11      22
31      12
30       8
3        7
4        7
12       5
10       3
5        3
6        1
13       1
15       1
44       1
32       1
43       1
dtype: int64

In [16]:
# 仅保留至少包含49个非NA值的行。
display(data.shape)
data.dropna(thresh=49,inplace=True)
# data.info()
data["status"].isnull().any()
display(data.shape)
data.isnull().sum(axis=1).value_counts()

(4754, 84)

(4476, 84)

0     3983
1      385
2       37
11      22
31      12
30       8
3        7
4        7
12       5
10       3
5        3
15       1
6        1
13       1
32       1
dtype: int64

In [17]:
data.info()
data.isnull().sum().argmax()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4476 entries, 0 to 4753
Data columns (total 84 columns):
low_volume_percent                            4474 non-null float64
middle_volume_percent                         4474 non-null float64
take_amount_in_later_12_month_highest         4476 non-null int64
trans_amount_increase_rate_lately             4473 non-null float64
trans_activity_month                          4474 non-null float64
trans_activity_day                            4474 non-null float64
transd_mcc                                    4474 non-null float64
trans_days_interval_filter                    4468 non-null float64
trans_days_interval                           4474 non-null float64
regional_mobility                             4474 non-null float64
student_feature                               4476 non-null float64
repayment_capability                          4476 non-null int64
is_high_user                                  4476 non-null int64
number_of_trans

/Users/shenqi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  


'cross_consume_count_last_1_month'

In [18]:
data["cross_consume_count_last_1_month"]

0       1.0
1       0.0
2       0.0
3       0.0
4       0.0
5       1.0
6       NaN
8       2.0
9       0.0
10      0.0
11      0.0
12      4.0
13      0.0
14      1.0
15      0.0
16      0.0
17      0.0
18      0.0
19      1.0
20      2.0
21      0.0
22      2.0
23      2.0
24      0.0
25      NaN
26      3.0
27      0.0
28      0.0
29      0.0
30      4.0
       ... 
4723    0.0
4724    0.0
4725    NaN
4726    0.0
4727    0.0
4728    0.0
4729    0.0
4730    0.0
4731    0.0
4732    NaN
4733    0.0
4734    0.0
4735    0.0
4736    0.0
4737    0.0
4738    3.0
4739    1.0
4740    1.0
4741    0.0
4742    0.0
4743    0.0
4744    0.0
4745    1.0
4747    3.0
4748    NaN
4749    0.0
4750    1.0
4751    1.0
4752    0.0
4753    1.0
Name: cross_consume_count_last_1_month, Length: 4476, dtype: float64

In [19]:
# 找出缺失值最多的特征，cross_consume_count_last_1_month，预估含义是用户超过一个月消费的累计次数，故考虑填充为0
data["cross_consume_count_last_1_month"].fillna(value=0,inplace=True)

In [20]:
# 其余缺失值考虑用均值来代替
data.fillna(data.mean(),inplace=True)

## 三. 数据切分

In [21]:
# 将标签和特征拆分
data_label = data["status"]
data.drop(["status"], axis=1, inplace=True)
data_features = data

,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,trans_days_interval,regional_mobility,student_feature,repayment_capability,is_high_user,number_of_trans_from_2011,first_transaction_time,historical_trans_amount,historical_trans_day,rank_trad_1_month,trans_amount_3_month,avg_consume_less_12_valid_month,abs,top_trans_count_last_1_month,avg_price_last_12_month,avg_price_top_last_12_valid_month,reg_preference_for_trad,trans_top_time_last_1_month,trans_top_time_last_6_month,consume_top_time_last_1_month,consume_top_time_last_6_month,cross_consume_count_last_1_month,trans_fail_top_count_enum_last_1_month,trans_fail_top_count_enum_last_6_month,trans_fail_top_count_enum_last_12_month,consume_mini_time_last_1_month,max_cumulative_consume_later_1_month,max_consume_count_later_6_month,railway_consume_count_last_12_month,pawns_auctions_trusts_consume_last_1_month,pawns_auctions_trusts_consume_last_6_month,jewelry_consume_count_last_6_month,first_transaction_day,trans_day_last_12_month,apply_score,apply_credibility,query_org_count,query_finance_count,query_cash_count,query_sum_count,latest_query_time,latest_one_month_apply,latest_three_month_apply,latest_six_month_apply,loans_score,loans_credibility_behavior,loans_count,loans_settle_count,loans_overdue_count,loans_org_count_behavior,consfin_org_count_behavior,loans_cash_count,latest_one_month_loan,latest_three_month_loan,latest_six_month_loan,history_suc_fee,history_fail_fee,latest_one_month_suc,latest_one_month_fail,loans_long_time,loans_latest_time,loans_credit_limit,loans_credibility_limit,loans_org_count_current,loans_product_count,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
0,0.01,0.99,0,0.90,0.55,0.313,17.0,27.0,26.0,3.0,0.0,19890,0,30.0,20130817.0,149050,151.0,0.40,34030,7.0,3920,0.15,1020,0.550000,1,4.0,19.0,4.0,19.0,1.0,1.0,2.0,2.0,5.0,2170,6.0,0.0,1970,18040,0.0,1738.0,85.0,583.000000,79.000000,8.000000,2.00,6.000000,10.000000,-383.000000,2.000000,5.00000,8.00000,552.0,73.0,37.0,34.0,2.0,10.0,1.0,9.0,1.0,1.0,13.0,37.0,7.0,1.0,0.0,341.0,-389.0,2200.0,72.0,9.0,10.0,2900.0,1688.0,1200.0,75.0,1.0,2.0,1200.0,1200.0,12.000000,18.0
1,0.02,0.94,2000,1.28,1.00,0.458,19.0,30.0,14.0,4.0,1.0,16970,0,23.0,20160402.0,302910,224.0,0.35,10590,5.0,6950,0.05,1210,0.500000,1,13.0,30.0,13.0,30.0,0.0,0.0,3.0,3.0,330.0,2100,9.0,0.0,1820,15680,0.0,779.0,84.0,653.000000,73.000000,7.000000,4.00,2.000000,8.000000,-375.000000,2.000000,6.00000,8.00000,635.0,76.0,37.0,36.0,0.0,17.0,5.0,12.0,2.0,2.0,8.0,49.0,4.0,2.0,1.0,353.0,-373.0,2000.0,74.0,12.0,12.0,3500.0,1758.0,15100.0,80.0,5.0,6.0,22800.0,9360.0,4.000000,2.0
2,0.04,0.96,0,1.00,1.00,0.114,13.0,68.0,22.0,1.0,0.0,9710,0,9.0,20170617.0,11520,31.0,1.00,5710,5.0,840,0.65,570,0.650000,1,0.0,68.0,0.0,68.0,0.0,3.0,6.0,6.0,0.0,0,3.0,0.0,0,0,0.0,338.0,95.0,654.000000,76.000000,11.000000,5.00,5.000000,16.000000,-373.000000,5.000000,5.00000,14.00000,633.0,83.0,4.0,2.0,0.0,3.0,1.0,2.0,2.0,2.0,4.0,2.0,2.0,1.0,1.0,157.0,-377.0,1500.0,77.0,2.0,2.0,1600.0,1250.0,4200.0,87.0,1.0,1.0,4200.0,4200.0,2.000000,6.0
3,0.00,0.96,2000,0.13,0.57,0.777,22.0,14.0,6.0,3.0,0.0,6210,0,33.0,20130516.0,491130,360.0,0.15,91690,7.0,46850,0.05,1290,0.450000,2,6.0,8.0,6.0,8.0,0.0,1.0,8.0,8.0,31700.0,8140,9.0,0.0,2700,27970,0.0,1831.0,82.0,595.000000,79.000000,12.000000,7.00,4.000000,22.000000,-373.000000,3.000000,16.00000,17.00000,542.0,75.0,85.0,81.0,4.0,22.0,5.0,17.0,2.0,4.0,34.0,91.0,26.0,2.0,0.0,355.0,-375.0,1800.0,74.0,17.0,18.0,3200.0,1541.0,16300.0,80.0,5.0,5.0,30000.0,12180.0,2.000000,4.0
4,0.01,0.99,0,0.46,1.00,0.175,13.0,66.0,42.0,1.0,0.0,11150,0,12.0,20170312.0,61470,63.0,0.65,9770,6.0,760,1.00,1110,0.500000,1,0.0,66.0,0.0,66.0,0.0,3.0,3.0,3.0,0.0,1000,3.0,0.0,0,6410,0.0,435.0,88.0,541.000000,75.

In [24]:
# 将数据按3:7拆分测试集和训练集
from sklearn.model_selection import train_test_split

data_features_train, data_features_test, data_label_train, data_label_test = train_test_split(data_features, data_label, test_size=0.3, random_state=2018)
display(data_features_train.shape)
display(data_features_test.shape)
display(data_label_train.shape)
display(data_label_test.shape)

(3133, 83)

(1343, 83)

(3133,)

(1343,)